# API Data Extraction for Financial Analysis

1. [What is API Data Extraction](#what-is-api-data-extraction)

2. [Types of Financial Data Available via APIs](#types-of-financial-data-available-via-apis)

3. [Data Quality and Bias Considerations](#data-quality-and-bias-considerations)

4. [How to Extract Data from MetaTrader](#how-to-extract-data-from-metatrader)

5. [How to Extract Data from yfinance](#how-to-extract-data-from-yfinance)

6. [How to Extract Data from Alpaca API](#how-to-extract-data-from-alpaca-api)

7. [Data Handling and Preprocessing](#data-handling-and-preprocessing)

8. [Summary and Best Practices](#summary-and-best-practices)


I will be putting information and examples while I develope my differents strategies

## What is API Data Extraction

It is the process of automatically retrieving data from external platforms or services through an API (Application Programming Interface), allowing access to financial information without manual downloads.
s.

---

## Types of Financial Data Available via APIs

APIs primarily provide access to **historical financial data**, which is essential for backtesting and analysis. This typically includes:

---

1. Historical price data (OHLCV – Open, High, Low, Close, Volume)

In [1]:
#EXAMPLE

2. Real-time or delayed market data (quotes, trades)

In [2]:
#EXAMPLE

3. Fundamental data (e.g., earnings, financial statements)

In [3]:
#EXAMPLE

4. Technical indicators (e.g., moving averages, RSI)

In [4]:
#EXAMPLE 

5. Alternative data (e.g., news sentiment, social media activity)

In [5]:
#EXAMPLE

Most APIs used in quantitative finance focus on **historical price series** for stocks, ETFs, forex, and cryptocurrencies.

---

## Data Quality and Bias Considerations

When working with financial data APIs, it's important to separate **data quality issues** from **modeling and bias-related treatments**. Below are two categories that highlight both aspects.

---

### A. Data Quality Issues

These concern the **accuracy, completeness, and consistency** of the raw data:

---

1. **Stock Splits**  
   If not adjusted, they can cause artificial price drops and mislead return calculations.  
   *Use APIs that provide adjusted historical prices or apply manual split adjustments.*


In [ ]:



# Initialize and login to MT5
mt5.initialize()
mt5.login(int(MT5_LOGIN), MT5_PWD, 'MetaQuotes-Demo')

# Fetch IGE data
from_date = datetime(2006, 4, 4)
to_date = datetime(2020, 4, 9)
symbol1 = "EWC"
symbol2 = "EWA"
symbol3 = "IGE"
timeframe = mt5.TIMEFRAME_H1


# ===========================================
# Fetch historical data for each symbol
# ===========================================
bars_ewc = mt5.copy_rates_range(symbol1, timeframe, from_date, to_date)
bars_ewa = mt5.copy_rates_range(symbol2, timeframe, from_date, to_date)
bars_ige = mt5.copy_rates_range(symbol3, timeframe, from_date, to_date)


# Convert to DataFrames
ewc = pd.DataFrame(bars_ewc)
ewa = pd.DataFrame(bars_ewa)
IGE = pd.DataFrame(bars_ige)

# Process dates
ewc['time'] = pd.to_datetime(ewc['time'], unit='s')
ewa['time'] = pd.to_datetime(ewa['time'], unit='s')
IGE['time'] = pd.to_datetime(IGE['time'], unit='s')

# Set the index as time
ewc.set_index('time', inplace=True)
ewa.set_index('time', inplace=True)
IGE.set_index('time', inplace=True)
IGE.sort_index(inplace=True)
IGE['close'] = IGE['close'].fillna(method='ffill').fillna(method='bfill')  # Fill NaNs

mt5.shutdown()



# Get common dates between all three series
common_dates = IGE.index.intersection(ewc.index).intersection(ewa.index)
IGE = IGE.loc[common_dates]
ewc = ewc.loc[common_dates]
ewa = ewa.loc[common_dates]

# Combine the series into a DataFrame
X = pd.concat([IGE['close'], ewc['close'], ewa['close']], axis=1)
X.columns = ['IGE', 'EWC', 'EWA']

#graph
plt.figure(figsize=(14, 6))
X.plot(ax=plt.gca())
plt.title('Price Series: IGE, EWC, EWA')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend(['IGE', 'EWC', 'EWA'])
plt.grid(True)
plt.tight_layout()
plt.show()
# Explanation of IGE's stock split in 2009
print("In 2009, the IGE investment fund underwent a stock split. This means that the number of shares in circulation increased, which affected its price, but not the total investment value. This split occurred specifically in May 2009.")






NameError: name 'mt5' is not defined

: 

2. **Stock/Bond Interest Payments**  
   Missing dividends or coupons distort total returns.  
   *Use total return series when available or combine price data with distribution info.*

In [ ]:
#EXAMPLE WILL BE PUT HERE

3. **Survivorship Bias**  
   Omits delisted or failed assets, overestimating strategy performance.  
   *Use survivorship-bias-free datasets that include inactive securities.*

In [ ]:
#EXAMPLE WILL BE PUT HERE

4. **Primary vs Consolidated Data**  
   Using only primary exchange data may miss trades from other venues.  
   *Use consolidated feeds for a full market view.*

In [ ]:
#EXAMPLE WILL BE PUT HERE

5. **Venue Currency & Market Fragmentation**  
   Data from different venues or currencies may be inconsistent or incomplete.  
   *Ensure currency normalization and use APIs with aggregated data when possible.*

In [ ]:
#EXAMPLE WILL BE PUT HERE


6. **Short Sale Restrictions & Illiquidity**  
   Historical restrictions on shorting or low liquidity can skew performance.  
   *Verify shorting availability and minimum liquidity thresholds.*

In [ ]:
#EXAMPLE WILL BE PUT HERE

7. **Futures Continuous Contracts & Adjustment Methods**  
   Creating continuous series may involve back-adjustments that affect signals.  
   *Be consistent with adjustment type (backward/forward/none) and strategy alignment.*

In [ ]:
#EXAMPLE WILL BE PUT HERE

8. **Futures Close Price: Settlement vs Last Traded**  
   Settlement prices may differ from the last traded, impacting daily PnL.  
   *Choose one method consistently and understand its implications for strategy logic.*

In [ ]:
 #EXAMPLE WILL BE PUT HERE

### B. Strategy and Bias Treatments

These relate to **how data is used** in strategy design and backtesting:

---

1. **Look-Ahead Bias**  
   Using future data for decision-making invalidates results.  
   *Always restrict data access to what would be known at each historical point.*

In [ ]:
#EXAMPLE WILL BE PUT HERE

2. **Data Snooping Bias**  
   Overfitting results by testing many strategies on the same dataset.  
   *Use out-of-sample testing, cross-validation, and penalize complexity.*

In [ ]:
#EXAMPLE WILL BE PUT HERE



3. **Strategy vs Long-Only Strategy**  
   Biases may affect long/short and long-only strategies differently.  
   *Compare both variants to evaluate robustness and detect overfitting.*

In [ ]:
#EXAMPLE WILL BE PUT HERE

# Download Data

Data can be downloaded from sources such as Yahoo Finance, Polygon, Metatrader...

---

1. **MetaTrader**  

 **Libraries**

In [ ]:
# ===========================================
# Import required libraries
# ===========================================

import pandas as pd  # For data manipulation and analysis (install: pip install pandas)
import numpy as np  # For numerical operations (install: pip install numpy)
import MetaTrader5 as mt5  # MT5 API to interact with the MetaTrader 5 terminal (install: pip install MetaTrader5)
from datetime import datetime  # To work with date/time ranges
import os  # For accessing environment variables
from dotenv import load_dotenv  # To load sensitive data (login, password) from .env file (install: pip install python-dotenv)

 **Log in**  

In [ ]:
# ===========================================
# Load environment variables from .env file
# ===========================================
# Create a .env file in the same folder and include:
# MT5_LOGIN=12345678
# MT5_PWD=your_password
# This helps protect your credentials.
load_dotenv()

MT5_LOGIN = os.getenv('MT5_LOGIN')
MT5_PWD = os.getenv('MT5_PWD')

# ===========================================
# Initialize and connect to MetaTrader 5
# ===========================================
mt5.initialize()

# Log in to your MT5 account (make sure MetaTrader 5 is running and connected)
# Replace 'MetaQuotes-Demo' with your broker's server if needed
mt5.login(int(MT5_LOGIN), MT5_PWD, 'MetaQuotes-Demo')


True

**Available assets**

In [ ]:
symbols = mt5.symbols_get()

nsymbols = 50
print(f"Total symbols found: {len(symbols)}")
for s in symbols[:nsymbols]:  # Print first 50 symbols
    print(s.name, "-", s.description)


#You can implement a search function to identify if it is avaible de asset you need

Total symbols found: 10466
AUDCAD - Australian Dollar vs Canadian Dollar
AUDCHF - Australian Dollar vs Swiss Franc
AUDDKK - Australian Dollar vs Danish Krona
AUDHKD - Australian Dollar vs Hong Kong Dollar
AUDHUF - Australian Dollar vs Hungarian Florint
AUDJPY - Australian Dollar vs Yen
AUDNOK - Australian Dollar vs Norwegian Krona
AUDNZD - Australian Dollar vs New Zealand Dollar
AUDPLN - Australian Dollar vs Zloty
AUDSEK - Australian Dollar vs Sweden Krona
AUDSGD - Australian Dollar vs Singapore Dollar
AUDTHB - Australian Dollar vs Thai Baht
AUDUSD - Australian Dollar vs US Dollar
AUDZAR - Australian Dollar vs South African Rand
CADCHF - Canadian Dollar vs Swiss Franc
CADDKK - Canadian Dollar vs Danish Krone
CADJPY - Canadian Dollar vs Yen
CADMXN - Canadian Dollar vs Mexican Peso
CADNOK - Canadian Dollar vs Norwegian Krona
CADPLN - Canadian Dollar vs Zloty
CADSEK - Canadian Dollar vs Swedish Krona
CADSGD - Canadian Dollar vs Singapore Dollar
CHFDKK - Swiss Franc vs Danish Krona
CHFHUF 

**Ticks & Bars**

In [ ]:
# ===========================================
# Define time range for data request
# ===========================================
from_date = datetime(2022, 1, 5)
to_date = datetime(2022, 1, 6)

# ===========================================
# Request tick data for a specific symbol (EURUSD)
# ===========================================
# You can choose COPY_TICKS_ALL, COPY_TICKS_TRADE, or COPY_TICKS_INFO
# COPY_TICKS_ALL returns both Bid and Ask ticks
ticks = mt5.copy_ticks_range("EURUSD", from_date, to_date, mt5.COPY_TICKS_ALL)

# ===========================================
# Create a pandas DataFrame to analyze the data
# ===========================================
data = pd.DataFrame(ticks)

# Convert 'time' from UNIX timestamp to readable datetime format
data['time'] = pd.to_datetime(data['time'], unit='s')

# Drop columns we don't need for now
data = data.drop(['last', 'volume', 'volume_real'], axis=1)

# Decode the 'flags' column (bitmask showing tick type: bid, ask, trade)
# Each flag is represented as an 8-bit binary string
data['flags'] = data['flags'].apply(lambda x: bin(x)[2:].zfill(8))

# Display the first few rows
print(data.head())

# ===========================================
# Example: How to request OHLC bars instead of ticks
# ===========================================
# Request 100 1-minute bars of EURUSD starting from a specific date:
bars = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_M1, datetime(2022, 1, 5), 100)

# Convert to DataFrame and format time
bars_df = pd.DataFrame(bars)
bars_df['time'] = pd.to_datetime(bars_df['time'], unit='s')

print(bars_df.head())

mt5.shutdown()

# ===========================================
# 📘 More info and documentation:
# Official MetaTrader 5 Python API docs:
# https://www.mql5.com/en/docs/python_metatrader5
# ===========================================

                 time      bid      ask       time_msc     flags
0 2022-01-04 23:00:00  1.12847  1.12848  1641337200067  10000010
1 2022-01-04 23:00:00  1.12845  1.12846  1641337200772  10000110
2 2022-01-04 23:00:00  1.12842  1.12843  1641337200907  10000110
3 2022-01-04 23:00:01  1.12842  1.12847  1641337201109  00000100
4 2022-01-04 23:00:01  1.12843  1.12848  1641337201733  10000110
                 time     open     high      low    close  tick_volume  \
0 2022-01-04 21:21:00  1.12865  1.12874  1.12864  1.12869           22   
1 2022-01-04 21:22:00  1.12872  1.12890  1.12872  1.12890           16   
2 2022-01-04 21:23:00  1.12890  1.12899  1.12885  1.12894           43   
3 2022-01-04 21:24:00  1.12894  1.12900  1.12894  1.12899           28   
4 2022-01-04 21:25:00  1.12899  1.12905  1.12898  1.12899           32   

   spread  real_volume  
0       1            0  
1       1            0  
2       1            0  
3       1            0  
4       1            0  


True

2. **Yahoo Finance**  




 **Libraries**

In [ ]:
# ===========================================
# 📦 Import required libraries
# ===========================================
import pandas as pd  # For data manipulation (install: pip install pandas)
import yfinance as yf  # Yahoo Finance API wrapper (install: pip install yfinance)
from datetime import datetime, timedelta  # To work with time ranges
from yahooquery import search  # For searching assets (install: pip install yahooquery)
import os  # To interact with the file system
import matplotlib.pyplot as plt  # For visualization (install: pip install matplotlib)
import json #For data manipulation


 **Search Assets**

In [ ]:
# ===========================================
# 🔍 1. Search for a ticker symbol
# ===========================================
query = "Tesla"  # Replace with any asset name, e.g., "Apple", "Bitcoin", etc.

# Use yahooquery to search for the asset
results = search(query)
# Convert in a list

tickers = []

for quote in results["quotes"]:
    tickers.append(quote["symbol"])

print(tickers)

NameError: name 'search' is not defined

**Ticks & Bars**

In [ ]:
# ===========================================
# ⏬ 3. Download intraday price data
# ===========================================
# You can modify 'interval' and 'period' as needed
# Available intervals: '1m', '2m', '5m', '15m', '1h', '1d', etc.
# Available periods: '1d', '5d', '1mo', '6mo', '1y', etc.

# Two ways to download information
# Download grouped data for all S&P 500 tickers
data = yf.download(
    tickers=tickers,
    interval='1m',
    period='1d',
    group_by='ticker',
    progress=True,
    threads=True  # Enable multi-threading for faster downloading
)

# Example: Download SPY data separately (for comparison or benchmarking)
spy = yf.download("SPY", interval='1m', period='1d')


data.head()
spy.head()

# ===========================================
# 🧠 Tips for further generalization
# ===========================================
# - You can adapt the script to download daily or weekly historical data
# - You could filter tickers by sector or industry if needed
# - Add error handling to skip failed downloads

# ===========================================
# 📘 Reference:
# yfinance documentation and GitHub:
# https://github.com/ranaroussi/yfinance
# ===========================================

NameError: name 'yf' is not defined

3. **Alpaca** 

---

 **Libraries**

In [ ]:
# ===========================================
# Import required libraries
# ===========================================

import os  # Access to environment variables
from dotenv import load_dotenv  # Load .env file variables
from alpaca.data.historical import stock  # Historical market data
from alpaca.trading.client import TradingClient  # Trading account access
from alpaca.data.requests import StockBarsRequest  # Request format for stock bars
from alpaca.data.timeframe import TimeFrame  # Time interval definitions
from datetime import datetime, date, timedelta  # Date and time handling
from alpaca.trading.requests import GetAssetsRequest  # For asset filtering
from alpaca.trading.enums import AssetStatus  # Enum to filter by asset status
import pandas as pd  # Data manipulation and display

 **Log in**  

In [ ]:
# Load environment variables from the .env file
load_dotenv()

# Get the API keys from the environment variables
api_key = os.getenv('ALPACA_KEY')
api_secret = os.getenv('ALPACA_SECRET')

# If API keys are not found, raise an error
if not api_key or not api_secret:
    raise EnvironmentError("Alpaca API environment variables are not set.")

# Alpaca API instance
api = stock.StockHistoricalDataClient(api_key=api_key, secret_key=api_secret)

# Instance client
client = TradingClient(api_key, api_secret, paper=True)

 **Search Assets**

In [ ]:

# Create asset request (e.g. only active assets)
request = GetAssetsRequest(status=AssetStatus.ACTIVE)

# Get list of assets
assets = client.get_all_assets(request)

# Number of assests
print(len(assets))

# Display first few assets
for asset in assets[:5]:
    print(asset.symbol, asset.name)

# Search for a specific asset by its symbol
symbol = "AAPL"
asset = trading_client.get_asset(symbol)

# Display basic info
print(f"Symbol: {asset.symbol}")
print(f"Name: {asset.name}")
print(f"Exchange: {asset.exchange}")
print(f"Tradable: {asset.tradable}")

12286
AUVPQ APPLIED UV INC NV PFD SER A
MOHOY ECMOHO LTD American Depositary Receipts - Sponsored (Cayman Islands)
DMSLQ DIGITAL MEDIA SOLUTIONS INC CL A NEW
ALSTF ALPHA STAR ACQUISITION CORP Rights   12/13/2026
OFED OCONEE FED FINL CORP Common Stock
Symbol: AAPL
Name: Apple Inc. Common Stock
Exchange: AssetExchange.NASDAQ
Tradable: True


**Ticks & Bars**

In [ ]:

# Define the ticker (e.g., 'AAPL' for Apple)
ticker = "AAPL"  # You can change this to the ticker of your preference
interval = "day"  # Time interval (can be 'minute', 'hour', 'day', 'week', 'month')
days = 30  # Number of historical days to fetch

# Assign the time interval based on the provided value
if interval == "minute":
    timeframe = TimeFrame.Minute
elif interval == "hour":
    timeframe = TimeFrame.Hour
elif interval == "day":
    timeframe = TimeFrame.Day
elif interval == "week":
    timeframe = TimeFrame.Week
elif interval == "month":
    timeframe = TimeFrame.Month
else:
    raise ValueError("Invalid interval. It should be 'minute', 'hour', 'day', 'week', or 'month'.")

# Define the start date
start = date.today() - timedelta(days=days)

# Request historical data for the specific ticker
request = StockBarsRequest(
    symbol_or_symbols=ticker,
    timeframe=timeframe,
    start=start,
    sort="desc"
)


def convert_to_dataframe(data):
    """
    Converts a list of tuples into a Pandas DataFrame with separated columns and timestamp as index.

    Args:
    - data (list): List of tuples containing historical data.

    Returns:
    - pd.DataFrame: DataFrame with appropriate columns.
    """
    # Create a list of dictionaries, where each dictionary represents a row.
    data_dict = []
    for item in data:
        row = {}
        for i in item:
            # Unpack the tuples and assign them to corresponding columns
            key, value = i
            row[key] = value
        data_dict.append(row)
    
    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(data_dict)
    
    # Convert the timestamp to index and ensure it is in datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    
    return df

try:
    # Fetch historical data
    chart_resp = api.get_stock_bars(request)
    
    # Check if the response contains data
    if chart_resp:
        # chart_resp[ticker] is a list of tuples
        data = chart_resp[ticker]  # List of tuples for the symbol
        
        # Convert the list of tuples into a DataFrame
        df = convert_to_dataframe(data)
        
        # Display the DataFrame
        print(df)
    else:
        print(f"No data found for {ticker}")
except Exception as e:
    print(f"Error fetching data for {ticker}: {e}")





                          symbol     open      high       low   close  \
timestamp                                                               
2025-05-01 04:00:00+00:00   AAPL  209.080  214.5600  208.9000  213.32   
2025-04-30 04:00:00+00:00   AAPL  209.300  213.5800  206.6705  212.50   
2025-04-29 04:00:00+00:00   AAPL  208.693  212.2400  208.3700  211.21   
2025-04-28 04:00:00+00:00   AAPL  210.000  211.5000  207.4600  210.14   
2025-04-25 04:00:00+00:00   AAPL  206.365  209.7500  206.2000  209.28   
2025-04-24 04:00:00+00:00   AAPL  204.890  208.8299  202.9400  208.37   
2025-04-23 04:00:00+00:00   AAPL  206.000  208.0000  202.7990  204.60   
2025-04-22 04:00:00+00:00   AAPL  196.120  201.5900  195.9700  199.74   
2025-04-21 04:00:00+00:00   AAPL  193.265  193.8000  189.8112  193.16   
2025-04-17 04:00:00+00:00   AAPL  197.200  198.8335  194.4200  196.98   
2025-04-16 04:00:00+00:00   AAPL  198.360  200.7000  192.3700  194.27   
2025-04-15 04:00:00+00:00   AAPL  201.855  203.5100

# Cointegration Test

You can use the **Engle-Granger** or **Johansen** test to check for cointegration between two time series.

---

# Modeling the Strategy

You will use techniques such as z-scores, Bollinger Bands, and linear regression to model your strategy.

---

# Strategy Results and interpretation

Evaluate the strategy's performance using metrics like returns, Sharpe ratio, and drawdowns.